In [255]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

import pandas as pd
pd.set_option('display.max_columns', 50)

import numpy as np
import pickle
import matplotlib.pyplot as plt
import time
import re


from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression as lm
from scipy import stats

import matplotlib.collections as mcol
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from matplotlib.lines import Line2D

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [172]:
df = pd.read_pickle("/home/seh6fy/git/publicrd/data/prd/Paper/FR_meta_and_final_tokens_23DEC21.pkl")


In [239]:
df = df[~df.DEPARTMENT.isnull()]

In [257]:
df.head()

,PROJECT_ID,ABSTRACT,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens
0,89996,"This is a project to explore Game-based, Metap...",Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",01,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN,1,1,1999467.0,1,project explore game base metaphor enhanced ga...
1,89997,Institution: Franklin Institute Science Museum...,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",02,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN,1,1,1799699.0,1,institution franklin institute science museum ...
2,89998,Through programs (including small group conver...,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",04,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN,1,1,1505858.0,1,program include small group conversation citiz...
3,89999,In partnership with the American Chemical Soci...,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",04,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN,1,1,51000.0,1,partnership american chemical society acs nati...
4,90001,The Center for Molecular Interfacing (CMI) wil...,Address; Architecture; Carbon Nanotubes; Catal...,CCI PHASE I: CENTER FOR MOLECULAR INTERFACING,NSF,NSF,NaN,0847926,10/1/2008,9/30/2011,"ABRUNA, HECTOR D",NaN,22,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.049,2008,1519821.0,NaN,1,1,1519821.0,1,center molecular interfacing cmi enable integr...


In [240]:
df_nsf = df.loc[df['DEPARTMENT'] == 'HHS']
df_nsf = df_nsf.iloc[0:10000,]


In [241]:
df_nasa = df.loc[df['DEPARTMENT'] == 'NASA']
df_nasa = df_nasa.iloc[0:10000,]

In [242]:
frames = [df_nsf, df_nasa]
df_split = pd.concat(frames)
df_split['DEPARTMENT']

13120      HHS
13121      HHS
13122      HHS
13123      HHS
13124      HHS
          ... 
477188    NASA
477189    NASA
477193    NASA
477205    NASA
477212    NASA
Name: DEPARTMENT, Length: 20000, dtype: object

In [243]:
df_split= df_split.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [244]:
# df= df.iloc[0:10000,]

In [245]:
# print(df)

In [258]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
training = v.fit_transform(df_split['ABSTRACT'])


In [259]:
# ids = df.index
# import random

In [260]:
training_data = df_split.sample(frac=0.95, random_state=25)
testing_data = df_split.drop(training_data.index)
# print(training_data)

In [261]:
train_id = training_data.index
test_id = testing_data.index

In [262]:
x_train= training[train_id]
x_test= training[test_id]

In [263]:
# type(test_id)

In [264]:
df_split['index'] = df_split.index
y_test = df_split.loc[df_split["index"].isin(test_id)]
y_test = y_test['DEPARTMENT']

In [265]:
df_split['index'] = df_split.index
y_train = df_split.loc[df_split["index"].isin(train_id)]
y_train = y_train['DEPARTMENT']
print(y_train)

0         HHS
1         HHS
2         HHS
3         HHS
4         HHS
         ... 
19994    NASA
19995    NASA
19996    NASA
19998    NASA
19999    NASA
Name: DEPARTMENT, Length: 19000, dtype: object


In [266]:
import math

In [267]:
neighbors = int(math.sqrt(len(df_split.index)))

In [268]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors= neighbors)
classifier.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=141)

In [269]:
y_pred = classifier.predict(x_test)

In [270]:
import scipy.sparse

In [271]:
# y_pred = pd.DataFrame(y_pred, columns = ['DEPARTMENT'])

In [272]:
predictions = df_split.loc[test_id,]
predictions['PREDICTIONS'] = y_pred

In [233]:
# predictions

In [273]:
predictions = predictions[["index","DEPARTMENT", "PREDICTIONS","ABSTRACT"]]

In [274]:
predictions 

,index,DEPARTMENT,PREDICTIONS,ABSTRACT
14,14,HHS,HHS,DESCRIPTION (provided by applicant): Primary...
32,32,HHS,HHS,DESCRIPTION (provided by applicant): The need ...
131,131,HHS,NASA,T cell activation and differentiation is depen...
133,133,HHS,NASA,DESCRIPTION (provided by applicant): The goal ...
144,144,HHS,HHS,DESCRIPTION: Smoking is an important public h...
...,...,...,...,...
19839,19839,NASA,NASA,GLOBAL DUST STORMS ARE A UNIQUELY MARTIAN ATMO...
19851,19851,NASA,NASA,"INTERPLANETARY SPACE IS FILLED WITH A TENUOUS,..."
19882,19882,NASA,NASA,DEVELOPMENT OF A HIGH-RESOLUTION GLOBAL SOIL D...
19965,19965,NASA,NASA,PRUNING THE TREE OF LIFE: EVOLUTION AND ECOLOG...


In [275]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[259 260]
 [230 251]]
              precision    recall  f1-score   support

         HHS       0.53      0.50      0.51       519
        NASA       0.49      0.52      0.51       481

    accuracy                           0.51      1000
   macro avg       0.51      0.51      0.51      1000
weighted avg       0.51      0.51      0.51      1000



In [276]:
error = []

# Calculating error for K values between 1 and 40
for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    pred_i = knn.predict(x_test)
    error.append(np.mean(pred_i != y_test))

[0.274,
 0.354,
 0.208,
 0.238,
 0.196,
 0.208,
 0.178,
 0.184,
 0.176,
 0.18,
 0.172,
 0.178,
 0.174,
 0.172,
 0.172,
 0.174,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.174,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172,
 0.172]

In [ ]:
#https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/

ValueError: cannot compute mean with no input